In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from household_package.data import call_data_url
from household_package.clean import clean_data
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from household_package.model import get_xy
from household_package.clean import filter_data
from household_package.registry import load_model
import pandas as pd
from household_package.registry import save_model
import pickle

In [98]:
BQ_DATASET

In [99]:
BUCKET_NAME="01_household_energy"
LOCAL_REGISTRY_PATH = "../model_h5/loaded/"


In [49]:
df_raw = call_data_url()

In [50]:
user_input = user_input = {
  "TYPEHUQ": 1,
  "NHSLDMEM": 1,
  "state_name": "Alabama",
  "REGIONC": "SOUTH",
  "BA_climate": "Hot-Humid",
  "SQFTEST": 240,
  "STORIES": 1,
  "YEARMADERANGE": 1,
  "NCOMBATH": 0,
  "NHAFBATH": 0,
  "TOTROOMS": 1,
  "WALLTYPE": 1,
  "ROOFTYPE": 1,
  "WINDOWS": 1,
  "SWIMPOOL": 1,
  "SOLAR": 1,
  "SMARTMETER": 1,
  "TELLWORK": 1,
  "DESKTOP": 0,
  "NUMLAPTOP": 0,
  "TVCOLOR": 0,
  "DISHWASH": 1,
  "MICRO": 0,
  "NUMFRIG": 0,
  "CWASHER": 1,
  "DRYER": 1,
  "LGTIN1TO4": 0,
  "LGTIN4TO8": 0,
  "LGTINMORE8": 0,
  "AIRCOND": 1,
  "EQUIPM": 3,
  "HEATHOME": 1,
  "NUMPORTEL": 0
}

In [51]:
def make_X_new(user_input):
    """
    This functions takes a dictionary coming from user inputs
    and makes a one row of X for model prediciton.
    """
    ## get all the passed arguments
    #user_inputs = locals().copy()
    X_new = pd.DataFrame({k:[v] for k,v in user_input.items()})
    return X_new


In [42]:
X_new = make_X_new(user_input)

In [28]:
#X_row_new = pd.DataFrame(pd.Series(user_input)).T

In [43]:
X_row_new = clean_data(X_new)

In [44]:
X_row_new

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,TYPEHUQ,NHSLDMEM,state_name,REGIONC,BA_climate,SQFTEST,...,MICRO,NUMFRIG,CWASHER,DRYER,AIRCOND,EQUIPM,HEATHOME,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,1,0,1,1,Mobile,1,Alabama,SOUTH,Hot-Humid,240,...,0,0,1,1,1,Furnace,1,0,0,0


In [59]:
from google.cloud import storage
client = storage.Client()
blobs = list(client.get_bucket(BUCKET_NAME).list_blobs(prefix="model"))

In [60]:
blobs

[<Blob: 01_household_energy, models/baseline_20231129-122043.pkl, 1701256844126884>,
 <Blob: 01_household_energy, models/baseline_20231129-131320.pkl, 1701260000941981>,
 <Blob: 01_household_energy, models/baseline_20231130-102940.pkl, 1701337090569311>]

In [101]:
latest_blob = max(blobs, key=lambda x: x.updated)
#print(latest_blob.name)
latest_model_path_to_save = os.path.join(LOCAL_REGISTRY_PATH, latest_blob.name)
print(latest_model_path_to_save)
latest_blob.download_to_filename(latest_model_path_to_save)
# for sklearn (baseline)
with open(latest_model_path_to_save , 'rb') as f:
    latest_model = pickle.load(f)

../model_h5/loaded/models/baseline_20231130-102940.pkl


In [70]:
latest_model.predict(X_row_new)

array([11040.])

In [3]:
model = load_model()

ValueError: Cannot determine path without bucket name.

In [31]:

#percentile_min = df_raw['KWH'].quantile(0.005)
#percentile_max = df_raw['KWH'].quantile(0.995)

# Filter the DataFrame based on the specified percentiles
#filtered_df = df_raw[(df_raw['KWH'] >= percentile_min) & (df_raw['KWH'] <= percentile_max)]
#filtered_df

In [18]:
df = clean_data(df_raw)

In [21]:
X , y = get_xy(df)

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size=0.3)

In [22]:
to_ohe_encode = ['REGIONC', 'state_name','BA_climate','TYPEHUQ',
                     'YEARMADERANGE','WALLTYPE','ROOFTYPE','WINDOWS','EQUIPM']
to_scale = ["NUMPORTEL", "STORIES","SQFTEST",
            "TOTROOMS", "NUMFRIG", "MICRO", "TVCOLOR","NHSLDMEM",
            "TOTAL_BATH", "TOTAL_COMP", "TOTAL_LIGHT" ]

min_max = MinMaxScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')


preprocessor = ColumnTransformer(transformers=[
                                            ('min_max', min_max, to_scale),
                                            ('ohe', ohe, to_ohe_encode)],
                                            remainder = "passthrough")

model = LinearRegression()

pipe = Pipeline([('prep', preprocessor), ('model', model)])

In [23]:
model = pipe.fit(X_train, y_train)

In [46]:
model.score(X_test,y_test)

0.5055383972861784

In [45]:
model.predict(X_row_new)

array([11760.])

In [34]:
X = df.drop('KWH',axis=1)

In [43]:
pd.DataFrame(X.iloc[0,:]).T

,SWIMPOOL,NUMPORTEL,SOLAR,SMARTMETER,REGIONC,state_name,BA_climate,TYPEHUQ,STORIES,YEARMADERANGE,...,TVCOLOR,TELLWORK,HEATHOME,EQUIPM,AIRCOND,NHSLDMEM,SQFTEST,TOTAL_BATH,TOTAL_COMP,TOTAL_LIGHT
0,0,0,0,0,WEST,New Mexico,Mixed-Dry,Single_detached,1,70-79,...,3,0,1,Furnace,1,2,2100,3,1,17


In [44]:
pipe.predict(pd.DataFrame(X.iloc[0,:]).T)

array([10640.])

ValueError: X does not contain any features, but ColumnTransformer is expecting 29 features